# Database Setup 

In [1]:
# Import libraries 
import requests
from pymongo import MongoClient
from datetime import datetime
import os
from typing import Dict, List, Any, Optional, Tuple
from pprint import pprint
from dataclasses import dataclass

from urllib.parse import urlparse
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pymongo import MongoClient
import json
import requests
from typing import Dict, List, Any

import logging
from pymongo import MongoClient, GEOSPHERE, UpdateOne
from pymongo.errors import BulkWriteError
import random 

In [2]:
# MongoDB connection
client = MongoClient('mongodb://mongodb:27017/')
db = client['sustainable_food_db']
collection = db['food_sources']

In [3]:
# This requires you to have an .env file in the folder directory that contains an API key 
API_KEY = os.getenv('USDA_API_KEY')

In [4]:
# API configuration
BASE_URLS = {
    'farmersmarket': 'https://www.usdalocalfoodportal.com/api/farmersmarket/',
    'csa': 'https://www.usdalocalfoodportal.com/api/csa/',
    'onfarmmarket': 'https://www.usdalocalfoodportal.com/api/onfarmmarket/'
}
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Referer": "https://www.google.com/"
}

In [5]:
@dataclass
class LocationQuery:
    """Represents possible location query parameters for the USDA API."""
    state: Optional[str] = None
    zip: Optional[str] = None
    city: Optional[str] = None
    coordinates: Optional[tuple[float, float]] = None
    radius: Optional[int] = None

    def validate(self) -> bool:
        """Validate that the query parameters follow USDA API rules."""
        if self.radius and self.radius > 100:
            return False
            
        if self.city and not self.state:
            return False
            
        # Must have at least one location parameter
        if not any([self.state, self.zip, self.city, self.coordinates]):
            return False
            
        return True

    def to_params(self) -> Dict[str, Any]:
        """Convert location query to API parameters."""
        params = {}
        
        if self.zip:
            params['zip'] = self.zip
            
        if self.city:
            params['city'] = self.city.lower()
        
        if self.state:
            params['state'] = self.state.lower()
            
        if self.coordinates:
            params['x'] = self.coordinates[0]
            params['y'] = self.coordinates[1]
            
        if self.radius:
            params['radius'] = min(self.radius, 100)  # Enforce maximum radius
            
        return params

In [6]:
class USDALocalFoodAPI:
    """Client for interacting with USDA Local Food Directory APIs."""
    
    BASE_URL = "https://www.usdalocalfoodportal.com/api"
    DIRECTORIES = {
        'csa': 'csa',
        'farmers_market': 'farmersmarket',
        'on_farm_market': 'onfarmmarket'
    }

    def __init__(self, api_key: str):
        self.api_key = api_key
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
            "Referer": "https://www.google.com/"
        }

    def _build_url(self, directory: str) -> str:
        """Build the API URL for a specific directory."""
        if directory not in self.DIRECTORIES:
            raise ValueError(f"Invalid directory. Must be one of: {', '.join(self.DIRECTORIES.keys())}")
        return f"{self.BASE_URL}/{self.DIRECTORIES[directory]}/"

    def fetch_listings(self, 
                      directory: str, 
                      location: LocationQuery,
                      max_retries: int = 3) -> List[Dict[str, Any]]:
        """
        Fetch listings from a specific directory using location parameters.
        
        Args:
            directory: Type of directory to query
            location: LocationQuery object containing search parameters
            max_retries: Maximum number of retry attempts for failed requests
            
        Returns:
            List of listing dictionaries
        """
        if not location.validate():
            raise ValueError("Invalid location query parameters")

        # Build request parameters
        params = location.to_params()
        params['apikey'] = self.api_key

        url = self._build_url(directory)
        
        # Implement retry logic
        for attempt in range(max_retries):
            try:
                response = requests.get(
                    url,
                    headers=self.headers,
                    params=params,
                    timeout=20
                )
                
                if response.status_code == 200:
                    return response.json().get('data', [])
                    
                elif response.status_code == 429:  # Rate limit
                    wait_time = min(2 ** attempt + random.uniform(0, 1), 60)  
                    logging.warning(f"Rate limited. Retrying in {wait_time:.2f} seconds...")
                    time.sleep(wait_time)
                    continue
                    
                else:
                    logging.error(f"API request failed: {response.status_code}")
                    response.raise_for_status()
                    
            except requests.exceptions.RequestException as e:
                if attempt == max_retries - 1:
                    logging.error(f"Failed to fetch data after {max_retries} attempts: {str(e)}")
                    raise
                continue

        return []

    def search_by_state(self, directory: str, state: str) -> List[Dict[str, Any]]:
        """Search listings by state."""
        query = LocationQuery(state=state)
        return self.fetch_listings(directory, query)

    def search_by_zip(self, 
                     directory: str, 
                     zip: str, 
                     radius: Optional[int] = None) -> List[Dict[str, Any]]:
        """Search listings by ZIP code with optional radius."""
        query = LocationQuery(zip=str(zip), radius=radius)
        return self.fetch_listings(directory, query)

    def search_by_city(self, directory: str, city: str, state: str) -> list:
        """
        Search for locations in a specific city and state.
        
        Args:
            directory: Type of directory to search (farmers_market, csa, on_farm_market)
            city: City name
            state: Two-letter state code
        """
        query = LocationQuery(city=city, state=state)
        if not query.validate():
            raise ValueError("Invalid city/state combination")
            
        response = self._make_request(
            f"{self.base_url}/{directory}s",
            params=query.to_params()
        )
        
        # Filter results to ensure they match the exact city
        if response and isinstance(response, list):
            filtered_results = [
                result for result in response 
                if result.get('city', '').lower() == city.lower() 
                and result.get('state', '').upper() == state.upper()
            ]
            return filtered_results
        
        return []

    def search_by_coordinates(self, 
                            directory: str, 
                            longitude: float, 
                            latitude: float, 
                            radius: Optional[int] = None) -> List[Dict[str, Any]]:
        """Search listings by coordinates with optional radius."""
        query = LocationQuery(coordinates=(longitude, latitude), radius=radius)
        return self.fetch_listings(directory, query)

In [7]:
api = USDALocalFoodAPI(api_key=API_KEY)

In [8]:
def parse_products(brief_desc: str) -> List[str]:
    """Extract product list from brief description."""
    if not brief_desc:
        return []
    
    products_section = brief_desc.split('Available Products:')[-1] if 'Available Products:' in brief_desc else ''
    products = [p.strip() for p in products_section.split(';') if p.strip()]
    return products
    
def parse_schedule(brief_desc: str) -> Dict[str, Any]:
    """Extract schedule information from brief description."""
    schedule = {
        "season_range": {
            "start_month": None,
            "end_month": None
        }
    }
    if brief_desc and 'Open:' in brief_desc:
        schedule_text = brief_desc.split('Open:')[1].split('<')[0].strip()
        months = schedule_text.split('to ')
        if len(months) == 2:
            schedule["season_range"]["start_month"] = months[0].replace(";", "").strip()
            schedule["season_range"]["end_month"] = months[1].replace(";", "").strip()
    return schedule
    
def transform_listing(listing: Dict[str, Any]) -> Dict[str, Any]:
    """Transform a single listing into the desired MongoDB format."""
    try:
        return {
            "directory_type": listing.get('directory_type'),
            "listing_id": listing.get('listing_id'),
            "listing_name": listing.get('listing_name'),
            "brief_desc": listing.get('brief_desc'),
            "updatetime": datetime.strptime(listing.get('updatetime', ''), '%b %dth, %Y') if listing.get('updatetime') else None,
            
            "contact": {
                "name": listing.get('contact_name'),
                "email": listing.get('contact_email'),
                "phone": listing.get('contact_phone')
            },
            
            "media": {
                "website": listing.get('media_website'),
                "facebook": listing.get('media_facebook'),
                "twitter": listing.get('media_twitter'),
                "instagram": listing.get('media_instagram')
            },
            
            "location": {
                "address": listing.get('location_address').title(),
                "state": listing.get('location_state').title(),
                "city": listing.get('location_city').title(),
                "street": listing.get('location_street').title(),
                "zipcode": listing.get('location_zipcode'),
                "coordinates": {
                    "type": "Point",
                    "coordinates": [
                        float(listing.get('location_x', 0)) if listing.get('location_x') else 0,
                        float(listing.get('location_y', 0)) if listing.get('location_y') else 0
                    ]
                }
            },
            
            "products": {
                "available_items": parse_products(listing.get('brief_desc')),
                "seasonal": True if listing.get('brief_desc') and 'seasonal' in listing.get('brief_desc').lower() else False
            },
            
            "schedule": parse_schedule(listing.get('brief_desc')),
            
            "metadata": {
                "last_sync": datetime.utcnow(),
                "verified": True
            }
        }
    except Exception as e:
        print(f"Error transforming listing {listing.get('listing_id')}: {e}")
        return None


## Reading in Data 

In [9]:
LocationQuery(city="Moorestown", state="NJ")

LocationQuery(state='NJ', zip=None, city='Moorestown', coordinates=None, radius=None)

In [11]:
TN_farmers_markets = api.search_by_state('farmers_market', 'TN')
print(f"Found {len(TN_farmers_markets)} farmers markets in Tennessee")

Found 94 farmers markets in Tennessee


In [17]:
states = [
    'AL',  # Alabama
    'Arkansas',  # Arkansas
    'FL',  # Florida
    'GA',  # Georgia
    'KY',  # Kentucky
    'Louisiana',  # Louisiana
    'MS',  # Mississippi
    'NC',  # North Carolina
    'SC',  # South Carolina
    'TN',  # Tennessee
    'Virginia'   # Virginia
]

# Types of food sources to search for
source_types = [
    'farmers_market',
    'on_farm_market',
    'csa'
]

# Initialize the nested dictionary to store results
southern_food_sources = {}

In [15]:
import time
import gc

In [18]:
# Gather data for each state
for state in states:
    southern_food_sources[state] = {}
    
    # For each type of food source
    for source_type in source_types:
        try:
            # Search API for this state and source type
            results = api.search_by_state(source_type, state)
            southern_food_sources[state][source_type] = results
            
            # Print progress update
            print(f"Retrieved {source_type} data for {state}")
            
        except Exception as e:
            print(f"Error retrieving {source_type} data for {state}: {str(e)}")
            southern_food_sources[state][source_type] = None
        time.sleep(1)
        gc.collect()

Retrieved farmers_market data for AL
Retrieved on_farm_market data for AL
Retrieved csa data for AL
Retrieved farmers_market data for Arkansas
Retrieved on_farm_market data for Arkansas
Retrieved csa data for Arkansas
Retrieved farmers_market data for FL
Retrieved on_farm_market data for FL
Retrieved csa data for FL
Retrieved farmers_market data for GA
Retrieved on_farm_market data for GA
Retrieved csa data for GA
Retrieved farmers_market data for KY
Retrieved on_farm_market data for KY
Retrieved csa data for KY
Retrieved farmers_market data for Louisiana
Retrieved on_farm_market data for Louisiana
Retrieved csa data for Louisiana
Retrieved farmers_market data for MS
Retrieved on_farm_market data for MS
Retrieved csa data for MS
Retrieved farmers_market data for NC
Retrieved on_farm_market data for NC
Retrieved csa data for NC
Retrieved farmers_market data for SC
Retrieved on_farm_market data for SC
Retrieved csa data for SC
Retrieved farmers_market data for TN
Retrieved on_farm_market

Upon manually inspecting the data,  I found that the the state abbrevations 'LA', 'VA' and 'AR' was getting data for Alabama, Nevada, and Arizona respectively which are not the states that correspond to each code. I found that typing the complete state name also works as a query parameter, so I fixed the entries for these states accordingly. 

In [14]:
#redo_states = ["Louisiana", "Virginia", "Arkansas"]

#for state in redo_states:
#    southern_food_sources[state] = {}
    
#    # For each type of food source
#    for source_type in source_types:
#        try:
#            # Search API for this state and source type
#            results = api.search_by_state(source_type, state)
#            southern_food_sources[state][source_type] = results
#            
#            # Print progress update
#            print(f"Retrieved {source_type} data for {state}")
#            
#        except Exception as e:
#            print(f"Error retrieving {source_type} data for {state}: {str(e)}")
#            southern_food_sources[state][source_type] = None

#del southern_food_sources['VA']
#del southern_food_sources['LA']
#del southern_food_sources['AR']

#southern_food_sources['VA'] = southern_food_sources.pop('Virginia')
#southern_food_sources['LA'] = southern_food_sources.pop('Louisiana')
#southern_food_sources['AR'] = southern_food_sources.pop('Arkansas')

In [19]:
len(southern_food_sources.keys()) == len(states)

True

In [42]:
collection.create_index([("location.coordinates", "2dsphere")])

'location.coordinates_2dsphere'

## Transform Data

In [20]:
def transform_listing_with_validation(listing: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """Transform a single listing with additional validation and error handling."""
    try:
        # Handle potentially missing or None values
        brief_desc = listing.get('brief_desc', '')
        update_time = listing.get('updatetime', '')
        
        # Helper function to safely apply title case
        def safe_title(value: Optional[str]) -> str:
            return value.title() if value else ''
        
        # Transform the listing using the existing function
        transformed = {
            "directory_type": listing.get('directory_type', ''),
            "listing_id": listing.get('listing_id', ''),
            "listing_name": listing.get('listing_name', ''),
            "brief_desc": brief_desc,
            "updatetime": parse_update_time(update_time),
            
            "contact": {
                "name": safe_title(listing.get('contact_name')),
                "email": listing.get('contact_email', ''),
                "phone": listing.get('contact_phone', '')
            },
            
            "media": {
                "website": clean_url(listing.get('media_website', '')),
                "facebook": clean_url(listing.get('media_facebook', '')),
                "twitter": clean_url(listing.get('media_twitter', '')),
                "instagram": clean_url(listing.get('media_instagram', ''))
            },
            
            "location": {
                "address": listing.get('location_address', ''),
                "state": safe_title(listing.get('location_state')),
                "city": safe_title(listing.get('location_city')),
                "street": safe_title(listing.get('location_street')),
                "zipcode": listing.get('location_zipcode', ''),
                "coordinates": {
                    "type": "Point",
                    "coordinates": [
                        float(listing.get('location_x', 0) or 0),
                        float(listing.get('location_y', 0) or 0)
                    ]
                }
            },
            
            "products": {
                "available_items": parse_products(brief_desc) if brief_desc else [],
                "seasonal": bool(brief_desc and 'seasonal' in brief_desc.lower())
            },
            
            "schedule": parse_schedule(brief_desc) if brief_desc else {},
            
            "metadata": {
                "last_sync": datetime.utcnow(),
                "verified": True
            }
        }
        
        return transformed
        
    except Exception as e:
        print(f"Error transforming listing {listing.get('listing_id')}: {e}")
        return None

def parse_update_time(update_time: str) -> Optional[datetime]:
    """Parse update time string into datetime object."""
    if not update_time:
        return None
        
    formats = [
        '%b %dth, %Y',
        '%b %dst, %Y',
        '%b %dnd, %Y',
        '%b %drd, %Y'
    ]
    
    for date_format in formats:
        try:
            return datetime.strptime(update_time, date_format)
        except ValueError:
            continue
            
    print(f"Could not parse update time: {update_time}")
    return None

def clean_url(url: str) -> str:
    """Clean and validate URL strings."""
    if not url:
        return ''
    
    # Remove common issues with URLs
    url = url.strip()
    if url.lower() == 'none':
        return ''
    
    # Ensure proper formatting for websites without http(s)
    if url and not url.startswith(('http://', 'https://', 'www.')):
        if url.startswith('//'):
            url = 'https:' + url
        elif not url.startswith('www.'):
            url = 'www.' + url
            
    return url

In [21]:
transformed_data = {}
for state, source_types in southern_food_sources.items():
    transformed_data[state] = []
    
    for source_type, listings in source_types.items():
        if not listings:  # Skip if None or empty
            continue
            
        for listing in listings:
            transformed = transform_listing_with_validation(listing)
            if transformed:
                transformed_data[state].append(transformed)

In [22]:
operations = []

for state_listings in transformed_data.values():
    for listing in state_listings:
        # Use listing_id as the unique identifier
        filter_doc = {"listing_id": listing["listing_id"]}
        
        # Add last_updated timestamp
        listing["metadata"]["last_updated"] = datetime.utcnow()
        
        # Create upsert operation
        operation = UpdateOne(
            filter_doc,
            {"$set": listing},
            upsert=True
        )
        
        operations.append(operation)

In [23]:
# Example of an operation
operations[10]

UpdateOne({'listing_id': '308278'}, {'$set': {'directory_type': 'farmersmarket', 'listing_id': '308278', 'listing_name': 'Albertville Farmers Market', 'brief_desc': '', 'updatetime': datetime.datetime(2023, 3, 20, 0, 0), 'contact': {'name': '', 'email': '', 'phone': '256-891-8202'}, 'media': {'website': '', 'facebook': '', 'twitter': '', 'instagram': ''}, 'location': {'address': '116 W MAIN ST, BERTVILLE, ALBERTVILLE, AL 35950', 'state': 'Alabama', 'city': 'Albertville', 'street': '116 W Main St, Bertville', 'zipcode': '35950', 'coordinates': {'type': 'Point', 'coordinates': [-86.20907036284322, 34.267951019308626]}}, 'products': {'available_items': [], 'seasonal': False}, 'schedule': {}, 'metadata': {'last_sync': datetime.datetime(2024, 11, 30, 22, 17, 37, 305970), 'verified': True, 'last_updated': datetime.datetime(2024, 11, 30, 22, 17, 37, 801287)}}}, True, None, None, None)

In [24]:
total_operations = len(operations)
total_operations

2128

## Create Database

In [26]:
db_name = 'sustainable_food_db',
collection_name = 'food_sources',
batch_size = 500

In [27]:
stats = {
    "total_processed": 0,
    "successful": 0,
    "failed": 0,
    "errors": []
}

In [28]:
collection

Collection(Database(MongoClient(host=['mongodb:27017'], document_class=dict, tz_aware=False, connect=True), 'sustainable_food_db'), 'food_sources')

In [29]:
batch_size = 500

for i in range(0, len(operations), batch_size):
    batch = operations[i:i + batch_size]
    
    try:
        result = collection.bulk_write(batch, ordered=False)
        stats["successful"] += result.modified_count + result.upserted_count
        
        logging.info(f"Processed batch {i//batch_size + 1}: "
                   f"{result.modified_count} modified, "
                   f"{result.upserted_count} upserted")
        
    except BulkWriteError as bwe:
        # Handle partial failures in batch
        stats["successful"] += bwe.details.get("nInserted", 0)
        stats["failed"] += len(bwe.details.get("writeErrors", []))
        stats["errors"].extend([
            {
                "index": error.get("index"),
                "code": error.get("code"),
                "message": error.get("errmsg")
            }
            for error in bwe.details.get("writeErrors", [])
        ])
        
        logging.error(f"Bulk write errors in batch {i//batch_size + 1}: "
                    f"{len(bwe.details.get('writeErrors', []))} errors")

In [30]:
stats

{'total_processed': 0, 'successful': 2128, 'failed': 0, 'errors': []}

In [31]:
summary = {
        "total_documents": collection.count_documents({}),
        "by_directory_type": {
            doc["_id"]: doc["count"]
            for doc in collection.aggregate([
                {"$group": {"_id": "$directory_type", "count": {"$sum": 1}}}
            ])
        },
        "by_state": {
            doc["_id"]: doc["count"]
            for doc in collection.aggregate([
                {"$group": {"_id": "$location.state", "count": {"$sum": 1}}}
            ])
        }
    }

In [32]:
summary

{'total_documents': 2128,
 'by_directory_type': {'onfarmmarket': 376, 'csa': 158, 'farmersmarket': 1594},
 'by_state': {'Mississippi': 60,
  'Tennessee': 155,
  'Florida': 322,
  'Georgia': 264,
  'Alabama': 135,
  'North Carolina': 274,
  'South Carolina': 120,
  'Virginia': 400,
  'Kentucky': 210,
  'Arkansas': 98,
  'Louisiana': 90}}

## Setting up State Area Database

In [33]:
# Read the CSV file
df = pd.read_csv('../us-states-by-area.csv')

# Create database
db = client['us_geography']

# Create collection
states_collection = db['states_area']

In [34]:
# Convert DataFrame to list of dictionaries
states_data = df.to_dict('records')

# Drop existing collection if it exists
states_collection.drop()

In [35]:
# Insert the data into MongoDB
states_collection.insert_many(states_data)

# Verify the data was inserted correctly
print(f"Number of documents inserted: {states_collection.count_documents({})}")

# Example query to check the data
print("\nExample document:")
print(states_collection.find_one())

Number of documents inserted: 52

Example document:
{'_id': ObjectId('674b8f5aebec22a43cbd95db'), 'state': 'Alabama', 'TotalArea': 52420.0, 'LandArea': 50645.0, 'WaterArea': 1775.0, 'densityMi': 101.5506}


In [36]:
# Create an index on state name for faster queries
states_collection.create_index("state")

'state_1'

## Setting up Seasonal Fruits and Vegetables Database

In [45]:
def process_list(veg_string):
    return sorted(set(item.strip() for item in veg_string.split(',')))

# Create the seasonal categories
seasonal_data = {
    "fall": process_list("""Acorn Squash, Asian Pear, Barbados Cherries, Black Crowberries, Black Salsify, 
    Belgian Endive, Broccoli, Brussels Sprouts, Butter Lettuce, Buttercup Squash, Butternut Squash, 
    Cactus Pear, Cape Gooseberries, Cardoon, Cauliflower, Chayote Squash, Chinese Long Beans, Crab Apples, 
    Cranberries, Date Plum, Delicata Squash, Diakon Radish, Endive, Feijoa, Garlic, Ginger, Grapes, Guava, 
    Hearts of Palm, Huckleberries, Jalapeno Peppers, Jerusalem Artichoke, Jujube, Key Limes, Kohlrabi, 
    Kumquats, Muscadine Grapes, Mushrooms, Ong Choy Spinach, Passion Fruit, Pear, Persimmons, Pineapple, 
    Pomegranate, Pumpkin, Quince, Radicchio, Sapote, Sharon Fruit, Sugar Apple, Sunflower Kernels, 
    Sweet Dumpling Squash, Sweet Potatoes, Swiss Chard, Turnips"""),
    
    "winter": process_list("""Belgian Endive, Brussels Sprouts, Buttercup Squash, Cactus Pear, Cardoon, 
    Cherimoya, Clementines, Collard Greens, Date Plums, Dates, Delicata Squash, Grapefruit, Kale, Kiwifruit, 
    Leeks, Mandarin Oranges, Maradol Papaya, Oranges, Passion Fruit, Pear, Persimmons, Pomegranate, Pummelo, 
    Red Banana, Red Currants, Sharon Fruit, Sweet Dumpling Squash, Sweet Potatoes, Tangerines, Turnips"""),
    
    "spring": process_list("""Apricots, Artichokes, Asparagus, Barbados Cherries, Belgian Endive, 
    Bitter Melon, Broccoli, Butter Lettuce, Cactus, Chayote Squash, Cherimoya, Chives, Collard Greens, 
    Corn, Fava Beans, Fennel, Fiddlehead Ferns, Green Beans, Honeydew, Jackfruit, Limes, Lychee, Mango, 
    Manoa Lettuce, Morel Mushrooms, Mustard Greens, Oranges, Pea Pods, Peas, Pineapple, Purple Asparagus, 
    Radicchio, Ramps, Red Leaf Lettuce, Rhubarb, Snow Peas, Sorrel, Spinach, Spring Baby Lettuce, 
    Strawberries, Swiss Chard, Vidalia Onions, Watercress, White Asparagus"""),
    
    "summer": process_list("""Anaheim Chile, Apricots, Armenian Cucumber, Asian Pear, Barbados Cherries, 
    Beets, Bell Peppers, Black Crowberries, Black Currants, Blackberries, Blueberries, Boysenberries, 
    Breadfruit, Butter Lettuce, Cantaloupe, Casaba Melon, Champagne Grapes, Chayote Squash, Cherries, 
    Cherries Sour, Chinese Long Beans, Corn, Crenshaw Melon, Crookneck Squash, Cucumbers, Durian, Eggplant, 
    Elderberries, Endive, French Beans, Figs, Galia Melon, Garlic, Grape Tomatoes, Grapefruit, Grapes, 
    Green Beans, Green Soybeans Edamame, Hearts of Palm, Honeydew Melons, Jackfruit, Jalapeno Peppers, 
    Key Limes, Lima Beans, Limes, Loganberries, Longan, Loquat, Lychee, Manoa Lettuce, Mulberries, 
    Nectarines, Okra, Ong Choy Spinach, Passion Fruit, Peaches, Peas, Persian Melon, Plums, Radishes, 
    Raspberries, Rose Apples, Sapodillas, Sapote, Shallots, Strawberries, Sugar Apple, Sugar Snap Peas, 
    Summer Squash, Tomatillo, Tomatoes, Watermelon, Winged Beans, Yukon Gold Potatoes, Zucchini"""),
    
    "year_round": process_list("""Amaranth, Apples, Arrowroot, Apricots Dried, Avocados, Bananas, 
    Banana Squash, Bell Peppers, Black Eyed Peas, Black Radish, Bok Choy, Broccoflower, Broccolini, 
    Burdock Root, Cabbage, Carrots, Cranberries Dried, Celeriac, Celery Root, Celery, Cherry Tomatoes, 
    Chinese Eggplants, Coconut, Dandelion Greens, Gai Lan, Galangal Root, Kiwano, Leek, Lemons, Lettuce, 
    Mushrooms, Olives, Onions, Papayas, Parsnips, Pearl Onions, Potatoes, Rutabagas, Salad Savoy, 
    Snow Peas, Wasabi Root, Yucca Root""")
}

In [46]:
db = client['food_database']
collection = db['seasonal_foods']

# Drop existing collection if it exists
collection.drop()

In [47]:
# Create documents for each food item
documents = []
for food in set().union(*seasonal_data.values()):
    seasons = []
    for season, foods in seasonal_data.items():
        if food in foods:
            seasons.append(season)
    
    documents.append({
        "name": food,
        "seasons": seasons,
        "available_year_round": "year_round" in seasons
    })

# Insert the documents
collection.insert_many(documents)

# Create indexes
collection.create_index("name")
collection.create_index("seasons")
collection.create_index("available_year_round")

'available_year_round_1'

In [48]:
print(f"Total foods in database: {collection.count_documents({})}")
print("\nSample year-round food:")
print(collection.find_one({"available_year_round": True}))
print("\nSample seasonal food:")
print(collection.find_one({"available_year_round": False}))

# Example aggregation to show foods available in multiple seasons
pipeline = [
    {"$match": {"available_year_round": False}},
    {"$project": {
        "name": 1,
        "season_count": {"$size": "$seasons"}
    }},
    {"$match": {"season_count": {"$gt": 1}}},
    {"$sort": {"season_count": -1}},
    {"$limit": 5}
]

print("\nFoods available in multiple seasons:")
for food in collection.aggregate(pipeline):
    print(f"{food['name']}: {food['season_count']} seasons")


Total foods in database: 192

Sample year-round food:
{'_id': ObjectId('674b99a1ebec22a43cbd96e8'), 'name': 'Amaranth', 'seasons': ['year_round'], 'available_year_round': True}

Sample seasonal food:
{'_id': ObjectId('674b99a1ebec22a43cbd96e9'), 'name': 'Spring Baby Lettuce', 'seasons': ['spring'], 'available_year_round': False}

Foods available in multiple seasons:
Belgian Endive: 3 seasons
Barbados Cherries: 3 seasons
Chayote Squash: 3 seasons
Butter Lettuce: 3 seasons
Passion Fruit: 3 seasons
